# Vue.js私房手册

## 安装和配置

- [windows环境搭建Vue开发环境](https://www.cnblogs.com/zhaomeizi/p/8483597.html)  
- [pycharm新建vue项目文件](https://www.jianshu.com/p/75258845c492)

## Vue CLI

- [Vue-CLI3使用mock](https://www.jianshu.com/p/37fef45c0381)

## Vue核心

## Vue Router

### 路由基本配置

特别注意：如果不是html5的history的模式，在浏览器里面写路由，或者在代码里面配置路由都是从`/#/`后面开始，一开始不知道，走很多弯路。

### 嵌套路由

嵌套路由要注意，预期说是嵌套路由，不如说是嵌套组件，首先子路由的`path`不加`/`是相对路由，接着父路由，如果加了`/`，就是绝对路由，是从根目录算起，如以下的路由配置：
```javascript
path: "/index",
component: () => import("layouts/MainLayout.vue"),
children: [
  {
    path: "/main",
    component: () => import("pages/Index.vue"),
  }
]
```
如果子路由是"/main"，则访问的时候的地址是`http://localhost:8080/#/main`。如果子路由的`path`是"main"，则访问地址为`http://localhost:8080/#/index/main`。但是不管那种`path`配置，浏览器都会先渲染父路由的组件，然后再渲染子路由的组件，因此都可以正常访问。

### 传参的几种方式

1. 使用`name`传值
在`route`配置中，指定了`route`的`name`以后，可以在组件中使用：
```html
<p>我是router-name: {{$route.name}}</p>
```
2. 使用`to`参数来传值
一般情况下，to参数是一个字符串，表示下一跳的路由，但是它也可以是对象，如：
```html
<router-link v-bind:to="{name:'xxx',params:{key:value}}"></router-link>
```
这里`name`要和定义路由时候配置的一样，使用的时候，语法如下：
```html
<p>传递的名字是：{{$route.params.username}}</p>
```
3. 使用url来传参
定义`route`的时候加冒号表示是变量，如下：
```javascript
{path: 'myjob/:jobid/:jobtitle', name='myjob', component:myjob}
```
在引用组件里面照常定义路由链接：
```html
<router link to="/myjob/12/web engineer">我的工作</router-link>
```
在组件里面，就可以通过绑定的变量来获取值：
```html
<a>jobid是:{{$route.params.jobid}}<a>
<a>jobtitle是:{{$route.params.jobtitle}}</a>
```
注意，url传参和`params`传参不能混用，也就是说有了`path`，就不能有`params`参数。

### 单页面多路由

一条路由不光可以对应一个组件，还可以注入多个组件，只要在定义路由的时候`component`参数是个对象，指定每个组件的名称，使用`<route view>`的时候标注注入的是哪个组件就行了。唯一要注意的是，注入时候没有指定名称参数的组件，引用的时候名称为`default`，如下：
```javascript
// 定义路由
components: {
  default:HelloWord,
  left:leftvue,
  right:rightvue
}
// 注入
<router-view/>
<router-view name="left"/>
<router-view name="right"/>
```

### 路由组件传参

普通的通过路由传参，需要在调用的组件里面通过`<div>User {{ $route.params.id }}</div>`的方式调用传递进来的参数，但是这样，如果是父组件调用这个组件，此时就会报错，组件就和路由绑定了。可以在编写路由的时候，通过`props`参数进行解绑。有3种方式：
1. 布尔模式
当`props=true`时，在组件里面就完全通过`props`进行调用，不用管传进来的参数是路由传递的还是父组件传递进来的。不过要注意的是，在组件里面，仍然可以使用`$route.params`进行调用的传递路由的动态参数。
2. 对象模式
这个情况下，只不过是给组件传递一个静态`props`，路由的动态参数在组件里面仍然只能使用`$route.params`的方式来调用。
3. 函数模式
此时函数是`(route)=>{}`的模式，`route`对象是自动传递到函数的参数。

要理解的是整个实现的过程，只要路由里面有动态参数，则会将`route.params`设置为组件的属性，然后自动将动态参数作为`props`参数传递给组件。

### 导航守卫

所谓导航守卫就是指路由发生变化的时候，自动触发的一些函数而已。要注意，可能在3个不同的地方触发函数，分别称为全局守卫、路由守卫以及组件内的守卫，这3个守卫分别定义在不同的地方。要理解不同的守卫，就要理解路由跳转的执行过程：
1. 导航被触发。
2. 在失活的组件里调用离开`beforeRouteLeave`守卫。<font color='red'>**全局**</font>
3. 调用全局的`beforeEach`守卫。<font color='red'>**全局**</font>
4. 在重用的组件里调用`beforeRouteUpdate`守卫 (2.2+)。<font color='red'>**组件，特别注意，查询参数的变化，即类似`get`请求的`/user?id=5`这种类型的路由会重用组件，不会重新渲染，因此不会调用组件内守卫`beforeRouteEnter`，此时会调用`beforeRouteUpdate`。注意，这个守卫函数里，组件实例已经被创建，`this`是可以使用的。**</font>
5. 在路由配置里调用`beforeEnter`。<font color='red'>**路由**</font>
6. 解析异步路由组件。
7. 在被激活的组件里调用`beforeRouteEnter`。<font color='red'>**组件，特别注意，在这个函数里不能调用组件实例`this`，因为它在渲染该组件的对应路由被`confirm`前调用，此时组件实例还没有创建。如果要用到组件，可以向`next`函数传递一个函数作为回调函数，函数的第一个参数就是组件实例，因为在第12步的时候，会调用这个回调函数，那时候组件就已经创建好了。**</font>
8. 调用全局的 beforeResolve 守卫 (2.5+)。<font color='red'>**全局,它和`router.beforeEach`的区别是在导航被确认之前，同时在所有组件内守卫和异步路由组件被解析之后，解析守卫就被调用。**</font>
9. 导航被确认。
10. 调用全局的 afterEach 钩子。<font color='red'>**全局，注意它不会接受`next`函数也不会改变导航本身。**</font>
11. 触发 DOM 更新。
12. 用创建好的实例调用`beforeRouteEnter`守卫中传给`next`的回调函数。

测试发现：
1. 如果组件没有重新渲染，比如动态参数，组件复用，比如增加或者改变查询参数，在浏览器里回车都不会触发`beforeRouteEnter`，但是如果点击刷新按钮，则都会触发`beforeRouteEnter`。而`beforeRouteUpdate`是只要路由发生了变化，则会触发一次，之后再刷新或者回车都不会触发。
2. 所有的守卫函数中，一定要调用`next`方法，否则不会`resolved`这个钩子（这里不太明白官网的`resolved`是什么意思，感觉应该是指解析下一个守卫函数）。测试的效果来看，如果不加`next`，就是执行完某个守卫函数以后就不再往下继续执行了，表现出来组件不会被渲染，都是空白，只有所有的守卫函数（个人理解就是官网的钩子）全部执行完，导航的状态才会变成`confirmed`（确认）。

## Vuex

### 通过参数来记忆`state`,`mutation`,`getter`和`action`

#### `state`

`state`相当于属性，定义的时候直接定义一个属性即可：
```javascript
state: {
    count: 0
}
```

在组件里面调用的时候把它放在计算属性里面，直接返回，如下：
```javascript
computed: {
    count () {
      return this.$store.state.count
    }
}
```
注意`mapState`辅助函数的写法，需要传入一个`state`的参数：
```javascript
mapState({
    count: state => state.count
})
```
如果是模块结构，比如`moduleA`模块的`state`，则这样写：
```javascript
mapState({
    count: state => state.moduleA.count
})
```
如果要获得局部状态，则必须使用常规函数：
```javascript
mapState({
    count(state){
        return state.count + this.localcount
    }
})
```

#### `mutation`

`mutation`相当于一个方法，它接受一个`state`和一个`payload`作为参数，主要对`state`进行操作，没有返回值，`payload`是可选的格外参数，名字可以随便取，注意，只能多传一个`payload`，如果想传多个参数的话打包成对象作为`payload`传入：
```javascript
mutations: {
    addcount(state，val) {
        state.count = state.count + val
    }
}
```
在组件里面调用的时候放在`method`里面，通过`store`的`commit`方法来调用`mutations`：
```javascript
method:{
    addcount(val){
        this.$store.commit('addcount', val)
    }
}
```
如果传递的不是一个值，而是一个对象，比如这样的定义：
```javascript
mutations: {
    addcount(state，payload) {
        state.count = state.count + payload.count
    }
}
```
调用的时候有两种语法：
```javascript
method:{
    addcount(val){
        this.$store.commit('addcount', {count: val})
    }
}
```
或者传给`commit`一个对象：
```javascript
method:{
    addcount(val){
        this.$store.commit({
            type: 'addcount',
            count: val
        })
    }
}
```

#### `getter`

`getter`可以理解为`store`的计算属性，也是一个方法，接受参数为`state`,`getters`，如果是模块形式，还可以接收2个参数为`rootState`和`rootGetters`，从而可以访问根节点的`state`和`getter`。注意，既然是计算属性，`getter`是有返回值的。
```javascript
getters:{
    newcount(state, getter, rootState, rootGetters){
        return state.count + rootState.count
    }
}
```
既然是计算属性，那么在组件里面调用的时候也是放在`computed`里面：
```javascript
computed: {
    newcount(){
        return this.$store.getters.newcount
    }
}
```
注意，如果是模块形式,比如这个`getter`是模块`moduleA`的`getter`，调用的方式是：
```javascript
return this.$store.getters["moduleA/newcount"]
```
注意，模块形式的`mapGetter`辅助函数的写法为：
```javascript
mapGetter({
    newcount: "moduleA/newcount"
})
```
和`state`不同，`getter`的`mapGetter`函数不需要传入任何参数。

#### `action`

`action`和`mutation`类似，也是一个方法，不过它是异步的，2点要注意：
1. `action`不能直接操作`state`，只能调用（也就是在`action`里面`commit`）`mutation`，主要是实现异步功能。
2. 它接收的第一个参数和`mutation`不一样，`mutation`第一个参数是`state`，而它是一个`context`，`context`类似一个扩展的`store`，第二个参数和`mutation`一样，也是一个`payload`。
```javascript
actions: {
  incrementAsync ({ commit }) {  //{ commit }是ES6的写法，直接提取`context`的`commit`方法
    setTimeout(() => {
      commit('increment')
    }, 1000)
  }
}
```

`action`通过`dispatch`触发，返回一个`promise`。因此可以这样使用：
```javascript
store.dispatch('actionA').then(() => {
  // ...
})
```

### 模块命名空间

默认情况下，模块内部的`action`、`mutation`和`getter`是注册在全局命名空间，如果设置`namespaced`属性为`true`，则在组件里面调用的时候需要指明路径。但是注意，`state`和前面几个都不一样，`state`有没有命名空间都一样，不存在路径一样的调用方式。

简单来说如下面的例子：
```javascript
//namespaced为false
import { mapMutations } from "vuex";

methods: {
    ...mapMutations({
      changecount: "moduleExample/otherMutation"
    })
}

//namespaced为true
import { mapMutations } from "vuex";

methods: {
    ...mapMutations({
      changecount: "otherMutation"
    })
}
```
而在组件里面调用`state`，只能象下面这样，不能`"moduleExample/subusername"`这样的方式调用：
```javascript
computed: mapState({
    username: "username", //username是根节点的state,"username"相当于state=>state.username
    subusername: state => state.moduleExample.subusername,
    count: state => state.moduleExample.count
})
```

## 问题收集

- [webpack@4x打包html里面img后src为“[object Module]”问题？](https://segmentfault.com/a/1190000021360248)
- [webpack@4x集成vueload@15x报`vue-loader was used without the corresponding plugin. Make sure to include VueLoaderPlugin in your webpack config.`错误](https://blog.csdn.net/cominglately/article/details/80555210)

## 经典代码收集

主要将官方的例子收集到一处方便查询：
- [动态组件官方基本例子](https://jsfiddle.net/2gadm0xt/1/)
- [动态组件`keep-alive`官方例子`](https://jsfiddle.net/chrisvfritz/Lp20op9o/)

## 相关教程

官方教程和重要资料：
- [Vue官网教程](https://cn.vuejs.org/v2/guide/)
- [所有事件名称查询](https://developer.mozilla.org/zh-CN/docs/Web/Events)
- [axios中文文档](https://www.kancloud.cn/yunye/axios/234845)

完整教程：
- [Vue框架Element UI+Mint UI教程汇总](https://www.jianshu.com/p/6295ee6974c9)
- [在pycharm中开发Vue快速入门](https://www.cnblogs.com/lizeqian1994/p/10690157.html)
- [Vue实战开发](https://blog.csdn.net/kevinfan2011/category_8319577.html)
- [Vue路由简明教程](https://my.oschina.net/u/3802541?tab=newest&catalogId=5761331)

知识点：
- [Vue生命周期总结](https://segmentfault.com/a/1190000008010666)
- [Vue登录拦截 登录后继续跳转指定页面](https://blog.csdn.net/qq_34576876/article/details/94356585)
- [Vue登录注册，并保持登录状态](https://segmentfault.com/a/1190000016040068)
- [在vue中使用session Storage和vuex保存用户登录状态](https://www.jianshu.com/p/48c912177167)
- [vuex存储和本地存储(localstorage、sessionstorage)的区别](https://www.cnblogs.com/jsanntq/p/9288144.html)
- [Vue.js中 watch 的高级用法](https://juejin.im/post/5ae91fa76fb9a07aa7677543)
- [谈谈axios配置请求头content-type](https://www.cnblogs.com/dreamcc/p/10752604.html)
- [vue.js - 奇怪的 event 对象](https://www.jianshu.com/p/b078cfe97c92)
- [Vue导入本地Js文件](https://www.jianshu.com/p/9e3e76ccf6e1)
- [Vue 中如何引入第三方 JS 库](https://blog.csdn.net/csdn_yudong/article/details/78795743)

## 引申阅读

- [一口气了解 babel](https://zhuanlan.zhihu.com/p/43249121)